In [1]:
import pandas as pd
#pd.set_option('display.max_rows', None)
import geopandas as gpd
import matplotlib.pyplot as plt
import os
import cartopy.crs as ccrs
import cartopy
import numpy as np
np.set_printoptions(threshold=33500)
from shapely.geometry import Polygon, Point, MultiPoint, MultiPolygon, GeometryCollection
from shapely.ops import cascaded_union
from datetime import datetime, timedelta
from collections import OrderedDict
import fiona
from matplotlib.path import Path
from timezonefinder import TimezoneFinder
import pytz

In [2]:
incidents = pd.read_csv('unique_fires.csv')
incidents = pd.concat([incidents, pd.DataFrame({'First Day UTC': np.zeros(len(incidents)),\
                                                'Last Day UTC': np.zeros(len(incidents)),\
                                               'Timezone': np.zeros(len(incidents))})], axis=1)
incidents = incidents[incidents['Fire Name']=='AUGUST COMPLEX'] #here we filter out August Complex
#incidents = incidents[(incidents['Fire Name']=='Williams Flats')| \
#                     (incidents['Fire Name']=='Shady')|\
#                     (incidents['Fire Name']=='Granite Gulch')|\
#                      (incidents['Fire Name']=='RIVERSIDE')|\
#                      (incidents['Fire Name']=='Holiday Farm')|\
#                     (incidents['Fire Name']=='204 Cow')|\
#                      (incidents['Fire Name']=='DOLAN')|\
#                      (incidents['Fire Name']=='BOBCAT')|\
#                      (incidents['Fire Name']=='PEDRO MOUNTAIN')|\
#                      (incidents['Fire Name']=='Cameron Peak')|\
#                      (incidents['Fire Name']=='EAST TROUBLESOME')|\
#                    (incidents['Fire Name']=='Walker')] #here we filter out some of Melinda's Fires
#incidents = incidents[incidents['Fire Name']=='Williams Flats'] #here we filter out August Complex

#print(incidents) # these are submitted in local time

#NEED TO DEAL WITH TIME ZONES, based on location and time of year, from here: https://pypi.org/project/pytz/
for ii in range(len(incidents)):
    obj=TimezoneFinder() #initialize the timezone finder
    tz = obj.timezone_at(lng=incidents['Lon Fire'].iloc[ii], lat=incidents['Lat Fire'].iloc[ii]) #get the timezone
    
    local = pytz.timezone(tz)
    utc = pytz.utc
    #put the start and end times in local time
    loc_dt_start = local.localize(datetime.strptime(incidents['First Day'].iloc[ii], '%Y-%m-%d %H:%M:%S'))
    loc_dt_end = local.localize(datetime.strptime(incidents['Last Day'].iloc[ii], '%Y-%m-%d'))
    #put them in UTC time
    utc_dt_start = loc_dt_start.astimezone(utc)
    utc_dt_end = loc_dt_end.astimezone(utc)
        
    #reassign to UTC time, this DOES keep track of daylight savings (eg +7 is used for PDT, +8 is used for PST)
    incidents['First Day UTC'].iloc[ii] = str(utc_dt_start)[0:19]
    incidents['Last Day UTC'].iloc[ii] = str(utc_dt_end)[0:19]
    incidents['Timezone'].iloc[ii] = tz    
incidents

,Incident Number,Fire Name,First Day,Last Day,Lat Fire,Lon Fire,First Day UTC,Last Day UTC,Timezone
1088,11843929.0,AUGUST COMPLEX,2020-08-17 13:44:00,2020-11-11,39.659444,-122.808889,2020-08-17 20:44:00,2020-11-11 08:00:00,America/Los_Angeles


In [3]:
viirs_all = pd.read_csv('merged_viirs.csv', dtype = {'satellite': str,'version': str, 'type': str})
viirs_all

,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight,type
0,46.145030,-114.230690,301.50,0.40,0.60,2019-12-04,842,1,VIIRS,n,2.0NRT,266.30,1.30,N,NaN
1,42.689530,-111.592030,309.10,0.55,0.51,2019-12-04,842,1,VIIRS,n,2.0NRT,270.90,8.00,N,NaN
2,47.817650,-103.056590,296.60,0.42,0.38,2019-12-04,842,1,VIIRS,n,2.0NRT,269.20,0.60,N,NaN
3,45.846410,-105.521350,297.80,0.47,0.40,2019-12-04,842,1,VIIRS,n,2.0NRT,272.00,0.80,N,NaN
4,45.842090,-105.516050,306.40,0.47,0.40,2019-12-04,842,1,VIIRS,n,2.0NRT,272.00,2.20,N,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1065362,39.142639,-121.509666,344.93,0.41,0.45,2020-12-31,2124,N,VIIRS,n,1,286.05,6.97,D,0.0
1065363,39.143078,-121.504829,328.83,0.41,0.45,2020-12-31,2124,N,VIIRS,n,1,284.42,4.39,D,0.0
1065364,38.063091,-121.160683,328.92,0.42,0.45,2020-12-31,2124,N,VIIRS,n,1,282.11,2.19,D,0.0
1065365,38.223763,-121.081375,325.87,0.42,0.46,2020-12-31,2124,N,VIIRS,n,1,280.99,2.84,D,0.0


In [4]:
#get the size of the resulting dataframe
df_size = 0
for ii in range(len(incidents)):
    print(ii)
    #start and end times of the fire in UTC
    inci_start = np.datetime64(incidents['First Day UTC'].iloc[ii])
    inci_end = np.datetime64(incidents['Last Day UTC'].iloc[ii]) 
    print(inci_start, inci_end)
    #how many overpass times are between the start and end time of this fire?
    inds_fire = np.where((viirs_all['acq_date'].iloc[:]>=str(inci_start))\
                                   & (viirs_all['acq_date'].iloc[:]<=str(inci_end)))[0]
    viirs_fire = viirs_all.iloc[inds_fire]
    dates = np.unique(viirs_fire['acq_date'])
    for day in dates:
        viirs_sub = viirs_fire.iloc[np.where(viirs_fire['acq_date']==day)[0]]
        df_size = df_size + len(np.unique(viirs_sub['acq_time']))
print(df_size)

0
2020-08-17T20:44:00 2020-11-11T08:00:00
1339


In [11]:
fire_series_new = gpd.GeoDataFrame({'Incident Number': np.nan*np.zeros(df_size), 
                          'Fire Name': np.nan*np.zeros(df_size), 
                         'Current Day': np.nan*np.zeros(df_size),
                        'Current Overpass': np.nan*np.zeros(df_size),
                                    'DateTime': np.nan*np.zeros(df_size),
                        'Lat Fire': np.nan*np.zeros(df_size), 
                     'Lon Fire': np.nan*np.zeros(df_size), 
                         'Number of NEW VIIRS points': np.nan*np.zeros(df_size), 
                                'NEW FRP': np.nan*np.zeros(df_size),
                'geometry': np.nan*np.zeros(df_size)}) # pre-allocate the dataframe
count = 0
for ii in range(len(incidents)):
    #start and end times of the fire in UTC
    inci_start = np.datetime64(incidents['First Day UTC'].iloc[ii])
    inci_end = np.datetime64(incidents['Last Day UTC'].iloc[ii]) 
    
    lat_fire = incidents.iloc[ii]['Lat Fire']
    lon_fire = incidents.iloc[ii]['Lon Fire']
    print(lon_fire, lat_fire)
    pt_start = Point(lon_fire, lat_fire)
    pt_start = pt_start.buffer(0.04)
    lat_old = np.array([lat_fire])
    lon_old = np.array([lon_fire])
    name = incidents.iloc[ii]['Fire Name']
    print(name)
    #how many overpass times are between the start and end time of this fire?
    inds_fire = np.where((viirs_all['acq_date'].iloc[:]>=str(inci_start))\
                                   & (viirs_all['acq_date'].iloc[:]<=str(inci_end)))[0]
    viirs_fire = viirs_all.iloc[inds_fire]
    dates = np.unique(viirs_fire['acq_date'])
    
    tic=np.datetime64('now')
    for day in dates[0:72]: #loop over the days 8/18 to 10/28
        viirs_day = viirs_fire.iloc[np.where(viirs_fire['acq_date']==day)[0]]
        overpasses = np.unique(viirs_day['acq_time'])
        for over in overpasses: #loop over the overpasses
            #print(day, over, count)
            
            viirs_over = viirs_day.iloc[np.where(viirs_day['acq_time']==over)[0]] # all points associated with an overpass
            lat = viirs_over['latitude'].values
            lon = viirs_over['longitude'].values
            viirs_tups = [list((lon[i], lat[i])) for i in range(len(lat))]
            #print(len(lat))
            
            if isinstance(pt_start, MultiPolygon):
                print('we have a multipolygon of length: ' + str(len(list(pt_start))))
            poly_today, poly_lats, poly_lons = build_today_polygon(pt_start, lat, lon, lat_old, lon_old, 0.5, 0)
            
            lat_cumulative = np.append(lat_old, poly_lats)
            lon_cumulative = np.append(lon_old, poly_lons)
            poly_tups_cumulative = [list((lon_cumulative[i], lat_cumulative[i])) for i in range(len(lon_cumulative))]

            poly_tups_new = [list((poly_lons[i], poly_lats[i])) for i in range(len(poly_lons))]
            
            #print(len(poly_tups_new))#, len(poly_tups_cumulative)) 
            
            if day==dates[0] and over==overpasses[0]: #if it's the first day of the fire
                #get the cumulative stuff 
                inds_new_points = [kk for kk in range(len(lat)) if viirs_tups[kk] in poly_tups_cumulative] #inds where we grab FRP
                frp =  np.sum(viirs_over['frp'].iloc[inds_new_points])
                #make a shape around the cumulative points, buffer by 300m (0.0027027 degrees ~0.003 degrees) NEED TO DIVIDE BY 2
                shape_new = MultiPoint(poly_tups_cumulative).buffer(0.0015625)#.convex_hull #the full shape so far
                shape_new_save = MultiPoint(poly_tups_cumulative).buffer(0.0015625)#.convex_hull #the full shape so far
                fire_series_new.iloc[count] = [incidents['Incident Number'].iloc[ii],\
                                       incidents['Fire Name'].iloc[ii],\
                                       day, over, day+'T'+str(over).zfill(4)[0:2]+':'+str(over).zfill(4)[2:4], \
                                    lat_fire, lon_fire, len(poly_tups_new),\
                                      frp, shape_new_save]     
                    
                count=count+1 
            elif len(poly_tups_new)!=0: #if we are adding new points
                inds_new_points = [kk for kk in range(len(lat)) if viirs_tups[kk] in poly_tups_new] #inds where we grab FRP
                frp =  np.sum(viirs_over['frp'].iloc[inds_new_points])
                
                shape_new = MultiPoint(poly_tups_new).buffer(0.0015625)
                #shape_cumu = MultiPoint(poly_tups_cumulative).buffer(0.0015625)
                shape_new_save = shape_new.difference(fire_series_new['geometry'].iloc[count-1]) #gets rid of overlaps
                #shape_new_save = shape_new.difference(shape_cumu)
                fire_series_new.iloc[count] = [incidents['Incident Number'].iloc[ii],\
                                       incidents['Fire Name'].iloc[ii],\
                                       day, over, day+'T'+str(over).zfill(4)[0:2]+':'+str(over).zfill(4)[2:4], \
                                               lat_fire, lon_fire, len(poly_tups_new),\
                                      frp, shape_new_save]     
                    
                count=count+1
            #re-assign for the next go round
            pt_start = shape_new
            #print(type(shape_new_save))
            lat_old = lat_cumulative
            lon_old = lon_cumulative
            print(count)
            
    toc=np.datetime64('now')
    print(toc-tic)
            
    #plot the entire incident
    fig, ax = plt.subplots(figsize=(20,8)) #set up the figure
    fire_series_new[fire_series_new['Fire Name']==name].plot(column ='Current Day',ax=ax,cmap='OrRd', legend=True,
                                                             legend_kwds={'loc': 'center left', 'bbox_to_anchor':(1,0.5)})
    plt.title(name + ' Thapa With Per Overpass Polygons')
    plt.show()
                
#drop the nans
fire_series_new = fire_series_new.dropna()
fire_series_new

-122.8088889 39.6594444
AUGUST COMPLEX
1
1
Time for recursive call
Time for recursive call
2
we have a multipolygon of length: 14
2
we have a multipolygon of length: 14
2
we have a multipolygon of length: 14
Time for recursive call
3
we have a multipolygon of length: 40
4
we have a multipolygon of length: 13
4
we have a multipolygon of length: 13
4
we have a multipolygon of length: 13
4
we have a multipolygon of length: 13
4
we have a multipolygon of length: 13
4
we have a multipolygon of length: 13
5
we have a multipolygon of length: 56
6
we have a multipolygon of length: 46
6
we have a multipolygon of length: 46
7
we have a multipolygon of length: 26
7
we have a multipolygon of length: 26
7
we have a multipolygon of length: 26
7
we have a multipolygon of length: 26
7
we have a multipolygon of length: 26
7
we have a multipolygon of length: 26
Time for recursive call
Time for recursive call
8
we have a multipolygon of length: 845
8
we have a multipolygon of length: 845
Time for recursi

KeyboardInterrupt: 

In [ ]:
datestr =day+'T'+str(over).zfill(4)[0:2]+':'+str(over).zfill(4)[2:4]
print(datestr)
print(np.datetime64(datestr))

In [ ]:
hi = fire_series_new['geometry']
hi
therest = fire_series_new[['Incident Number', 'Fire Name', 'Current Day','Current Overpass','Lat Fire', 'Lon Fire','Number of NEW VIIRS points', 'NEW FRP']]
therest

fiona.supported_drivers
hi.to_file("august_complex_polygons_only_VIIRS_overpass.geojson", driver='GeoJSON')
therest.to_csv('august_complex_polygons_attributes_VIIRS_overpass.csv')

In [5]:
#lat_old and lon_old are the detects used to build the prev version of today's polygon
def build_today_polygon(shape, lats, lons, lats_old, lons_old, bf, plot_evol):
    shape_buff = shape.buffer(bf)
    #find the points that are inside the buffer
    pairs = zip(lons, lats)
    lat_new = []
    lon_new = []
    if isinstance(shape_buff, Polygon):
        xs, ys = shape_buff.exterior.xy #get the edges of the buffered shape
        poly_path = Path(np.stack([xs, ys], axis=1)) #make them into a path
        poly_path_contains = poly_path.contains_points(np.stack([lons,lats],axis=1)) #get the points that are in the shape REPLACES FOR LOOP
        #grab the new points
        lat_new =lats[np.where(poly_path_contains==True)[0]]
        lon_new =lons[np.where(poly_path_contains==True)[0]]
    elif isinstance(shape_buff, MultiPolygon):    
        for geom in list(shape_buff): #for when the buffered geometry is a multipolygon
            xs, ys = geom.exterior.xy #get the edges of the buffered shape
            poly_path = Path(np.stack([xs, ys], axis=1)) #make them into a path
            poly_path_contains = poly_path.contains_points(np.stack([lons,lats],axis=1)) #get the points that are in the shape REPLACES FOR LOOP
            #grab the new points
            lat_new = np.append(lat_new, lats[np.where(poly_path_contains==True)[0]])
            lon_new = np.append(lon_new, lons[np.where(poly_path_contains==True)[0]])

    #make a shape around the new points
    tups = [list((lon_new[i], lat_new[i])) for i in range(len(lat_new))]
    mp = MultiPoint(tups).convex_hull
    
    
    if plot_evol == 1: #plot the new polygon
        print(len(lat_new), len(lon_new))
        m = gpd.GeoSeries(mp)
        m.plot()
        plt.scatter(lon_new, lat_new, c = 'k')
        plt.scatter(lon_fire, lat_fire, c='r')
        plt.title('Updated shape and points')
        plt.show()
        
    if (len(lat_new)-len(lats_old))>0: #if we are adding points 
        print('Time for recursive call')
        return build_today_polygon(mp, lats, lons, lat_new, lon_new, bf, plot_evol)
    else:
        return mp, lat_new, lon_new #MAY WANT TO RETURN A BUFFERED VERSION OF THE POLYGON
    

In [ ]:
print(dates[0:72])

In [ ]:
#inci_start = np.datetime64(incidents['First Day'].iloc[0]) + np.timedelta64(7,'h')
#inci_end = np.datetime64(incidents['Last Day'].iloc[0]) + np.timedelta64(7,'h')

                #plot it up
                #fig = plt.figure(figsize=(15,10))
                #ax= fig.add_subplot(111,projection=ccrs.PlateCarree())
                ##ax.set_extent([-124, -122, 39,41])
                #ax.set_extent([-123, -122.5, 39.5, 40])
                ##gpd.GeoSeries(shape_cumulative).plot(ax=ax, color='b', alpha = 0.5)
                #gpd.GeoSeries(shape_new).plot(ax=ax,color='r',alpha=0.75 )#
               ## gpd.GeoSeries(mp).plot(ax=ax, color='gray', alpha = 0.15)
                #gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                #      linewidth=2, color='gray', alpha=0.5, linestyle='--')
                #gl.top_labels = False
                #gl.right_labels = False
                #plt.title(str(day)+str(over))
                #plt.show()

In [ ]:
a = Point(1, 1).buffer(1.5)
b = Point(2, 1).buffer(1.5)
a